# SQLite 불러오기

In [ ]:
import sqlite3


def connect_example_db():
    try:
        conn = sqlite3.connect('northwind.db')
        cur = conn.cursor()
    except sqlite3.Error as e:
        print('Err:', e)

    return conn, cur

conn, cur = connect_example_db()

In [ ]:
from IPython.display import display, Markdown


def execute_and_show(cur: sqlite3.Cursor, query: str, *args):
    '''execute the query and show the result with markdown'''
    data = cur.execute(query, args)
    md_str_list = []

    if data.description is None:
        return
    
    column_names = list(map(lambda column: column[0], data.description))
    md_str_list.append('|' + '|'.join(column_names) + '|')
    md_str_list.append('|' + '-|' * len(column_names))
    for row in data:
        row = list(map(lambda cell: '' if cell is None else str(cell), row))
        md_str_list.append('|' + '|'.join(row) + '|')

    md_str = '\n'.join(md_str_list)
    md = Markdown(md_str)
    display(md)

# SQL SELECT Statement

In [ ]:
# select some columns
execute_and_show(cur, 'SELECT CompanyName, City FROM Customers;')

In [ ]:
# select all columns
execute_and_show(cur, 'SELECT * FROM Customers;')

# SQL SELECT DISTINCT

In [ ]:
# select distinct
execute_and_show(cur,
                 'SELECT DISTINCT Country FROM Customers;')

In [ ]:
# select without distinct
execute_and_show(cur,
                 'SELECT Country FROM Customers;')

In [ ]:
# count distinct
execute_and_show(cur,
                 'SELECT COUNT(DISTINCT Country) FROM Customers;')

In [ ]:
# count distinct for MS Access
execute_and_show(cur,
                 """
                 SELECT Count(*) AS DistinctCountries
                 FROM (SELECT DISTINCT Country FROM Customers);
                 """)

- SQLite에서 `Count(Distinct Countries)`와 `Count(*)` 결과값이 다른 이유
    - `Count(Distinct Countries)`
        - 집계시 NULL 제외
    - `Count(*)`
        - NULL 상관 없이 row 수로 집계

출처: https://www.sqlite.org/lang_aggfunc.html

# SQL WHERE

In [ ]:
# where statement for text field
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE Country='Mexico';
                 ''')

In [ ]:
# where statement for numeric field
execute_and_show(cur,
                 '''
                 SELECT * FROM Products
                 WHERE ProductID=1;
                 ''')

In [ ]:
# operators in the WHERE clause
execute_and_show(cur,
                 '''
                 SELECT * FROM Products
                 WHERE ProductID > 50;
                 ''')

# SQL ORDER BY

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Products
                 ORDER BY UnitPrice;
                 ''')

In [ ]:
# ORDER BY DESC
execute_and_show(cur,
                 '''
                 SELECT * FROM Products
                 ORDER BY UnitPrice DESC;
                 ''')

In [ ]:
# ordering alphabetically
execute_and_show(cur,
                 '''
                 SELECT * FROM Products
                 ORDER BY ProductName;
                 ''')

In [ ]:
# ordering alphabetically DESC
execute_and_show(cur,
                 '''
                 SELECT * FROM Products
                 ORDER BY ProductName DESC;
                 ''')

In [ ]:
# ORDER BY several columns
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 ORDER BY Country, CompanyName
                 ''')

In [ ]:
# using both ASC and DESC
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 ORDER BY Country ASC, CompanyName DESC;
                 ''')

# SQL AND operator

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT *
                 FROM Customers
                 WHERE Country='Spain' AND CompanyName LIKE 'G%';
                 ''')

In [ ]:
# all conditions must be true
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE Country='Germany'
                 AND City='Berlin'
                 AND PostalCode > 12000;
                 ''')

# SQL OR Operator

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT *
                 FROM Customers
                 WHERE Country='Germany' OR Country='Spain';
                 ''')

In [ ]:
# at least one condition must be true
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE City = 'Berlin' OR CompanyName LIKE 'G%' OR Country='Norway';
                 ''')

In [ ]:
# And + OR
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE Country='Spain' AND (CompanyName LIKE 'G%' OR CompanyName LIKE 'R%');
                 ''')

In [ ]:
# AND first, OR follows
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE Country='Spain' AND CompanyName LIKE 'G%' OR CompanyName LIKE 'R%';
                 ''')

# SQL NOT Operator

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE NOT Country='Spain';
                 ''')

In [ ]:
# NOT LIKE
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE CompanyName NOT LIKE 'A%';
                 ''')

In [ ]:
# NOT BETWEEN
execute_and_show(cur,
                 '''
                 SELECT * FROM Products
                 WHERE UnitPrice NOT BETWEEN 10 AND 60;
                 ''')

In [ ]:
# NOT IN
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE City NOT IN ('Paris', 'London');
                 ''')

In [ ]:
# NOT Grater than
execute_and_show(cur,
                 '''
                 SELECT * FROM Products
                 WHERE NOT UnitPrice > 20;
                 ''')

In [ ]:
# NOT Less than
execute_and_show(cur,
                 '''
                 SELECT * FROM Products
                 WHERE NOT UnitPrice < 20;
                 ''')

# SQL INSERT INTO

In [ ]:
execute_and_show(cur,
                 '''
                 INSERT INTO Customers (CustomerID, CompanyName, ContactName, Address, City, PostalCode, Country)
                 VALUES ('CARDL', 'Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway');
                 ''')

In [ ]:
# insert data only in specified columns
execute_and_show(cur,
                 '''
                 INSERT INTO Customers (CompanyName, City, Country)
                 VALUES ('Cardinal', 'Stavanger', 'Norway');
                 ''')

In [ ]:
# insert multiple rows
execute_and_show(cur,
                 '''
                 INSERT INTO Customers (CompanyName, ContactName, Address, City, PostalCode, Country)
                 VALUES
                 ('Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway'),
                 ('Greasy Burger', 'Per Olsen', 'Gateveien 15', 'Sandnes', '4306', 'Norway'),
                 ('Tasty Tee', 'Finn Egan', 'Streetroad 19B', 'Liverpool', 'L1 0AA', 'UK');
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 ORDER BY CustomerID LIMIT 5;
                 ''')

# SQL NULL Values

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT CompanyName, ContactName, Address
                 FROM Customers
                 WHERE Address IS NULL;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT CompanyName, ContactName, Address
                 FROM Customers
                 WHERE Address IS NOT NULL;
                 ''')

# SQL UPDATE

In [ ]:
execute_and_show(cur,
                 '''
                 UPDATE Customers
                 SET ContactName='Alfred Schmidt', City='Frankfurt'
                 WHERE CustomerID='ALFKI';
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE CustomerID='ALFKI';
                 ''')

In [ ]:
# update multple records
execute_and_show(cur,
                 '''
                 UPDATE Customers
                 SET ContactName='Juan'
                 WHERE Country='Mexico';
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE Country='Mexico';
                 ''')

In [ ]:
# update all rows
execute_and_show(cur,
                 '''
                 UPDATE Customers
                 SET ContactName='Juan';
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT CustomerID, ContactName FROM Customers
                 ''')

In [ ]:
# close updated database
conn.close()

In [ ]:
# restore database
!rm northwind.db
!cp northwind_original.db northwind.db

In [ ]:
# reconnect database
conn, cur = connect_example_db()

# SQL DELETE

In [ ]:
execute_and_show(cur,
                 '''
                 DELETE FROM Customers WHERE CompanyName='Alfreds Futterkiste';
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE CompanyName='Alfred Futterkiste';
                 ''')

In [ ]:
# delete all rows
execute_and_show(cur,
                 '''
                 DELETE FROM Customers;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers;
                 ''')

In [ ]:
# delete table
execute_and_show(cur,
                 '''
                 DROP TABLE Customers;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers;
                 ''')

In [ ]:
# close updated database
conn.close()

In [ ]:
# restore database
!rm northwind.db
!cp northwind_original.db northwind.db

In [ ]:
# reconnect database
conn, cur = connect_example_db()

# SQL TOP, LIMIT, FETCH FIRST or ROWNUM

- `TOP`
    - SQL Server, MS Access 지원
- `LIMIT`
    - MySQL, SQLite 지원
- `FETCH FIRST`
    - Oracle 12 지원
- `ROWNUM`
    - Oracle 버전 지원
    - 12 버전 이전 방식

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 LIMIT 3;
                 ''')

In [ ]:
# ADD a WHERE clause
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE Country='Germany'
                 LIMIT 3;
                 ''')

In [ ]:
# add the ORDER BY keyword
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 ORDER BY CompanyName DESC
                 LIMIT 3;
                 ''')

# SQL MIN and MAX

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT MIN(UnitPrice)
                 FROM Products;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT MAX(UnitPrice)
                 FROM Products;
                 ''')

In [ ]:
# set column name
execute_and_show(cur,
                 '''
                 SELECT MIN(UnitPrice) AS SmallestPrice
                 FROM Products;
                 ''')

In [ ]:
# use MIN with GROUP BY
execute_and_show(cur,
                 '''
                 SELECT MIN(UnitPrice) AS SmallestPrice, CategoryID
                 FROM Products
                 GROUP BY CategoryID;
                 ''')

# SQL COUNT

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT COUNT(*)
                 FROM Products;
                 ''')

In [ ]:
# specify column
execute_and_show(cur,
                 '''
                 SELECT COUNT(ProductName)
                 FROM Products;
                 ''')

In [ ]:
# add a WHERE clause
execute_and_show(cur,
                 '''
                 SELECT COUNT(ProductID)
                 FROM Products
                 WHERE UnitPrice > 20;
                 ''')

In [ ]:
# ignore duplicates
execute_and_show(cur,
                 '''
                 SELECT COUNT(DISTINCT UnitPrice)
                 FROM Products;
                 ''')

In [ ]:
# use an alias
execute_and_show(cur,
                 '''
                 SELECT COUNT(*) AS [Number of records]
                 FROM Products;
                 ''')

In [ ]:
# use COUNT with GROUP BY
execute_and_show(cur,
                 '''
                 SELECT COUNT(*) AS [Number of records], CategoryID
                 FROM Products
                 GROUP BY CategoryID;
                 ''')

# SQL SUM

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT SUM(Quantity)
                 FROM [Order Details];
                 ''')

In [ ]:
# add a WHERE clause
execute_and_show(cur,
                 '''
                 SELECT SUM(Quantity)
                 FROM [Order Details]
                 WHERE ProductID=11;
                 ''')

In [ ]:
# use an alias
execute_and_show(cur,
                 '''
                 SELECT SUM(Quantity) AS total
                 FROM [Order Details];
                 ''')

In [ ]:
# use SUM with GROUP BY
# 결과가 너무 많은 관계로 일부 생략
execute_and_show(cur,
                 '''
                 SELECT OrderID, SUM(Quantity) AS [Total Quantity]
                 FROM [Order Details]
                 GROUP BY OrderID
                 LIMIT 20;
                 ''')

In [ ]:
# SUM with an expression
execute_and_show(cur,
                 '''
                 SELECT SUM(Quantity * 10)
                 FROM [Order Details];
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT SUM(Products.UnitPrice * Quantity)
                 FROM [Order Details]
                 LEFT JOIN Products ON [Order Details].ProductID = Products.ProductID;
                 ''')

# SQL AVG

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT AVG(UnitPrice)
                 FROM Products;
                 ''')

In [ ]:
# add a WHERE clause
execute_and_show(cur,
                 '''
                 SELECT AVG(UnitPrice)
                 FROM Products
                 WHERE CategoryID = 1;
                 ''')

In [ ]:
# use an Alias
execute_and_show(cur,
                 '''
                 SELECT AVG(UnitPrice) AS [average price]
                 FROM Products;
                 ''')

In [ ]:
# higher than average
execute_and_show(cur,
                 '''
                 SELECT * FROM Products
                 WHERE UnitPrice > (SELECT AVG(UnitPrice) FROM Products);
                 ''')

In [ ]:
# use AVG with GROUP BY
execute_and_show(cur,
                 '''
                 SELECT AVG(UnitPrice) AS AveragePrice, CategoryID
                 FROM Products
                 GROUP By CategoryID;
                 ''')

# SQL LIKE

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE CompanyName LIKE 'a%';
                 ''')

In [ ]:
# _ wildcard
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE City LIKE 'L_nd__';
                 ''')

In [ ]:
# % wildcard
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE City LIKE '%L%';
                 ''')

In [ ]:
# starts with
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE CompanyName LIKE 'La%';
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE CompanyName LIKE 'a%' OR CompanyName LIKE 'b%';
                 ''')

In [ ]:
# ends with
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE CompanyName LIKE '%a';
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE CompanyName LIKE 'b%s';
                 ''')

In [ ]:
# contains
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE CompanyName LIKE '%or%';
                 ''')

In [ ]:
# combine wildcard
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE CompanyName LIKE 'a__%';
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE CompanyName LIKE '_r%';
                 ''')

In [ ]:
# without wildcard
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE Country LIKE 'Spain';
                 ''')

# SQL Wildcard

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE CompanyName LIKE 'a%';
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE CompanyName LIKE '%es';
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE CompanyName LIKE '%mer%';
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE City LIKE '_ondon';
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE City LIKE 'L___on';
                 ''')

# SQL IN

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE Country IN ('Germany', 'France', 'UK');
                 ''')

In [ ]:
# NOT IN
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE Country NOT IN ('Germany', 'France', 'UK');
                 ''')

In [ ]:
# IN (SELECT)
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE CustomerID IN (SELECT CustomerID FROM Orders);
                 ''')

In [ ]:
# NOT IN (SELECT)
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers
                 WHERE CustomerID NOT IN (SELECT CustomerID FROM Orders);
                 ''')

# SQL BETWEEN

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Products
                 WHERE UnitPrice BETWEEN 10 AND 20;
                 ''')

In [ ]:
# NOT BETWEEN
execute_and_show(cur,
                 '''
                 SELECT * FROM Products
                 WHERE UnitPrice NOT BETWEEN 10 AND 20;
                 ''')

In [ ]:
# BETWEEN with IN
execute_and_show(cur,
                 '''
                 SELECT * FROM Products
                 WHERE UnitPrice BETWEEN 10 AND 20
                 AND CategoryID IN (1, 2, 3);
                 ''')

In [ ]:
# BETWEEN text values (alphabetical order)
execute_and_show(cur,
                 '''
                 SELECT * FROM Products
                 WHERE ProductName BETWEEN 'Carnarvon Tigers' AND 'Mozzarella di Giovanni'
                 ORDER BY ProductName;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Products
                 WHERE ProductName BETWEEN "Carnarvon Tigers" AND "Chef Anton's Cajun Seasoning"
                 ORDER BY ProductName;
                 ''')

In [ ]:
# NOT BETWEEN text values
execute_and_show(cur,
                 '''
                 SELECT * FROM Products
                 WHERE ProductName NOT BETWEEN 'Carnarvon Tigers' AND 'Mozzarella di Giovanni'
                 ORDER BY ProductName;
                 ''')

In [ ]:
# BETWEEN dates
execute_and_show(cur,
                 '''
                 SELECT * FROM Orders
                 WHERE OrderDate BETWEEN '2016-07-01' AND '2016-07-31';
                 ''')

# SQL Aliases

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT CustomerID AS ID
                 FROM Customers;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT CustomerID ID
                 FROM Customers;
                 ''')

In [ ]:
# alias for columns
execute_and_show(cur,
                 '''
                 SELECT CustomerID AS ID, CompanyName AS Customer
                 FROM Customers;
                 ''')

In [ ]:
# using aliases with a space character
execute_and_show(cur,
                 '''
                 SELECT ProductName AS [My Great Products]
                 FROM Products;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT ProductName AS "My Great Products"
                 FROM Products;
                 ''')

In [ ]:
# concatenate columns
execute_and_show(cur,
                 '''
                 SELECT CompanyName, Address || ', ' || PostalCode || ' ' || City || ', ' || Country AS Address
                 FROM Customers;
                 ''')

In [ ]:
# alias for tables
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers AS Persons;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT o.OrderID, o.OrderDate, c.CompanyName
                 FROM Customers AS c, Orders AS o
                 WHERE c.CompanyName='Around the Horn' AND c.CustomerID=o.CustomerID;
                 ''')

In [ ]:
# same result as above, but without aliases
execute_and_show(cur,
                 '''
                 SELECT Orders.OrderID, Orders.OrderDate, Customers.CompanyName
                 FROM Customers, Orders
                 WHERE Customers.CompanyName='Around the Horn' AND Customers.CustomerID=Orders.CustomerID;
                 ''')

# SQL JOIN

In [ ]:
# 결과가 너무 큰 관계로 일부 생략
execute_and_show(cur,
                 '''
                 SELECT Orders.OrderID, Customers.CompanyName, Orders.OrderDate
                 FROM Orders
                 INNER JOIN Customers ON Orders.CustomerID=Customers.CustomerID
                 LIMIT 20;
                 ''')

# SQL INNER JOIN

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT ProductID, ProductName, CategoryName
                 FROM Products
                 INNER JOIN Categories ON Products.CategoryID=Categories.CategoryID;
                 ''')

In [ ]:
# naming the columns
execute_and_show(cur,
                 '''
                 SELECT Products.ProductID, Products.ProductName, Categories.CategoryName
                 FROM Products
                 INNER JOIN Categories ON Products.CategoryID=Categories.CategoryID;
                 ''')

In [ ]:
# JOIN or INNER JOIN
execute_and_show(cur,
                 '''
                 SELECT Products.ProductID, Products.ProductName, Categories.CategoryName
                 FROM Products
                 JOIN Categories ON Products.CategoryID = Categories.CategoryID;
                 ''')

In [ ]:
# JOIN three tables
# 결과가 너무 큰 관계로 일부 생략
execute_and_show(cur,
                 '''
                 SELECT Orders.OrderID, Customers.CompanyName, Shippers.CompanyName
                 FROM ((Orders
                 INNER JOIN Customers ON Orders.CustomerID = Customers.CustomerID)
                 INNER JOIN Shippers ON Orders.ShipVia = Shippers.ShipperID)
                 LIMIT 20;
                 ''')

# SQL LEFT JOIN

In [ ]:
# 결과가 너무 많은 관계로 일부 생략
execute_and_show(cur,
                 '''
                 SELECT Customers.CompanyName, Orders.OrderID
                 FROM Customers
                 LEFT JOIN Orders ON Customers.CustomerID = Orders.CustomerID
                 ORDER BY Customers.CompanyName
                 LIMIT 20;
                 ''')

# SQL RIGHT JOIN

In [ ]:
# 결과가 많은 관계로 일부 생략
execute_and_show(cur,
                 '''
                 SELECT Orders.OrderID, Employees.LastName, Employees.FirstName
                 FROM Orders
                 RIGHT JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID
                 ORDER BY Orders.OrderID
                 LIMIT 20;
                 ''')

# SQL FULL OUTER JOIN

In [ ]:
# 결과가 너무 많은 관계로 일부 생략
execute_and_show(cur,
                 '''
                 SELECT Customers.CompanyName, Orders.OrderID
                 FROM Customers
                 FULL OUTER JOIN Orders ON Customers.CustomerID = Orders.CustomerID
                 ORDER BY Customers.CompanyName
                 LIMIT 20;
                 ''')

# SQL Self Join

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT A.CompanyName AS CustomerName1, B.CompanyName AS CustomerName2, A.City
                 FROM Customers A, Customers B
                 WHERE A.CustomerID <> B.CustomerID
                 AND A.City = B.City
                 ORDER BY A.City;
                 ''')

# SQL UNION

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT City FROM Customers
                 UNION
                 SELECT City FROM Suppliers
                 ORDER BY City;
                 ''')

In [ ]:
# UNION ALL
execute_and_show(cur,
                 '''
                 SELECT City FROM Customers
                 UNION ALL
                 SELECT City FROM Suppliers
                 ORDER BY City;
                 ''')

In [ ]:
# UNION with WHERE
execute_and_show(cur,
                 '''
                 SELECT City, Country FROM Customers
                 WHERE Country='Germany'
                 UNION
                 SELECT City, Country FROM Suppliers
                 WHERE Country='Germany'
                 ORDER BY City;
                 ''')

In [ ]:
# UNION ALL with WHERE
execute_and_show(cur,
                 '''
                 SELECT City, Country FROM Customers
                 WHERE Country='Germany'
                 UNION ALL
                 SELECT City, Country FROM Suppliers
                 WHERE Country='Germany'
                 ORDER BY City;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT 'Customer' AS Type, ContactName, City, Country
                 FROM Customers
                 UNION
                 SELECT 'Supplier', ContactName, City, Country
                 FROM Suppliers;
                 ''')

# SQL GROUP BY

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT COUNT(CustomerID), Country
                 FROM Customers
                 GROUP BY Country;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT COUNT(CustomerID), Country
                 FROM Customers
                 GROUP BY Country
                 Order BY COUNT(CustomerID) DESC;
                 ''')

In [ ]:
# GROUP BY with JOIN
execute_and_show(cur,
                 '''
                 SELECT Shippers.CompanyName, COUNT(Orders.OrderID) AS NumberOfOrders FROM Orders
                 LEFT JOIN Shippers ON Orders.ShipVia = Shippers.ShipperID
                 GROUP BY CompanyName;
                 ''')

# SQL HAVING

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT COUNT(CustomerID), Country
                 FROM Customers
                 GROUP BY Country
                 HAVING COUNT(CustomerID) > 5;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT COUNT(CustomerID), Country
                 FROM Customers
                 GROUP BY Country
                 HAVING COUNT(CustomerID) > 5
                 ORDER BY COUNT(CustomerID) DESC;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT Employees.LastName, COUNT(Orders.OrderID) AS NumberOfOrders
                 FROM (Orders
                 INNER JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID)
                 GROUP BY LastName
                 HAVING COUNT(Orders.OrderID) > 10;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT Employees.LastName, COUNT(Orders.OrderID) AS NumberOfOrders
                 FROM Orders
                 INNER JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID
                 WHERE LastName = 'Davolio' OR LastName = 'Fuller'
                 GROUP BY LastName
                 HAVING COUNT(Orders.OrderID) > 25;
                 ''')

# SQL EXISTS

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT CompanyName
                 FROM Suppliers
                 WHERE EXISTS (SELECT ProductName FROM Products WHERE Products.SupplierID = Suppliers.SupplierID AND UnitPrice < 20);
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT CompanyName
                 FROM Suppliers
                 WHERE EXISTS (SELECT ProductName FROM Products WHERE Products.SupplierID = Suppliers.SupplierID AND UnitPrice = 22);
                 ''')

# SQL ANY and ALL

- `ANY`
    - SQLite에서는 미지원

In [ ]:
# ProductID = ANY ... 대신 IN 연산자 사용
execute_and_show(cur,
                 '''
                 SELECT ProductName
                 FROM Products
                 WHERE ProductID IN
                    (SELECT ProductID
                    FROM [Order Details]
                    WHERE Quantity = 10);
                 ''')

In [ ]:
# ProductID = ANY ... 대신 IN 연산자 사용
execute_and_show(cur,
                 '''
                 SELECT ProductName
                 FROM Products
                 WHERE ProductID IN
                    (SELECT ProductID
                    FROM [Order Details]
                    WHERE Quantity > 99);
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT ProductName
                 FROM Products
                 WHERE ProductID IN
                    (SELECT ProductID
                    FROM [Order Details]
                    WHERE Quantity > 1000);
                 ''')

In [ ]:
# 일반 SELECT문과 동일
execute_and_show(cur,
                 '''
                 SELECT ALL ProductName
                 FROM Products
                 WHERE True;
                 ''')

In [ ]:
# SQLite는 ALL를 지원하지 않아 다른 방식으로 대체
execute_and_show(cur,
                 '''
                 SELECT ProductName, ProductID
                 FROM Products P
                 WHERE 
                    (SELECT COUNT(ProductID) FROM [Order Details] O WHERE O.Quantity = 10)
                     = (SELECT COUNT(ProductID) FROM [Order Details] O WHERE O.Quantity = 10 AND O.ProductID <= P.ProductID);
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT MAX(ProductID) FROM [Order Details] O WHERE O.Quantity = 10;
                 ''')

# SQL SELECT INTO

In [ ]:
execute_and_show(cur,
                 '''
                 CREATE TABLE CustomersBackup2017 AS SELECT * FROM Customers;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM CustomersBackup2017 LIMIT 20;
                 ''')

In [ ]:
# SQLite에서 외부 데이터베이스의 테이블을 불러올 때는 SELECT INTO IN 문 대신 ATTACH DATABASE 사용
execute_and_show(cur,
                 '''
                 ATTACH DATABASE 'northwind_bak.db' AS external_db;
                 ''')

execute_and_show(cur,
                 '''
                 CREATE TABLE ProductsBackup AS SELECT * FROM external_db.Products;
                 ''')

execute_and_show(cur,
                 '''
                 DETACH DATABASE external_db;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM ProductsBackup LIMIT 20;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 DROP TABLE CustomersBackup2017;
                 ''')

In [ ]:
# SELECT INTO with specific columns
execute_and_show(cur,
                 '''
                 CREATE TABLE CustomersBackup2017 AS SELECT CompanyName, ContactName FROM Customers;
                 ''')

execute_and_show(cur,
                 '''
                 SELECT * FROM CustomersBackup2017 LIMIT 20;
                 ''')

In [ ]:
# with WHERE clause
execute_and_show(cur,
                 '''
                 CREATE TABLE CustomersGermany
                 AS SELECT * FROM Customers
                 WHERE Country = 'Germany';
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM CustomersGermany LIMIT 20;
                 ''')

In [ ]:
# with JOIN clause
execute_and_show(cur,
                 '''
                 CREATE TABLE CustomersOrdersBackup2017
                 AS SELECT Customers.CompanyName, Orders.OrderID
                 FROM CustomerS
                 LEFT JOIN Orders ON Customers.CustomerID = Orders.CustomerID;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM CustomersOrdersBackup2017 Limit 20;
                 ''')

In [ ]:
# create empty table with another schema
execute_and_show(cur,
                 '''
                 CREATE TABLE NewTable
                 AS SELECT *
                 FROM Customers
                 WHERE 1 = 0;
                 ''')

execute_and_show(cur,
                 '''
                 SELECT * FROM NewTable;
                 ''')

# SQL INSERT INTO SELECT

In [ ]:
execute_and_show(cur,
                 '''
                 INSERT INTO Customers (CompanyName, City, Country)
                 SELECT CompanyName, City, Country FROM Suppliers;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 INSERT INTO Customers (CompanyName, ContactName, Address, City, PostalCode, Country)
                 SELECT CompanyName, ContactName, Address, City, PostalCode, Country FROM Suppliers;
                 ''')

In [ ]:
# with WHERE clause
execute_and_show(cur,
                 '''
                 INSERT INTO Customers (CompanyName, City, Country)
                 SELECT CompanyName, City, Country FROM Suppliers
                 WHERE Country = 'Germany';
                 ''')

# SQL CASE

In [ ]:
# use CASE as column
execute_and_show(cur,
                 '''
                 SELECT OrderId, Quantity,
                 CASE
                    WHEN Quantity > 30 THEN 'The quantity is greater than 30'
                    WHEN Quantity = 30 THEN 'The quantity is 30'
                    ELSE 'The quantity is under 30'
                 END AS QuantityText
                 FROM [Order Details]
                 LIMIT 20;
                 ''')

In [ ]:
# with ORDER BY
execute_and_show(cur,
                 '''
                 SELECT CompanyName, City, Country
                 FROM Customers
                 ORDER BY
                 (CASE
                    WHEN City IS NULL THEN Country
                    ELSE City
                 END)
                 LIMIT 30;
                 ''')

# SQL NULL

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT ProductName, UnitPrice * (UnitsInStock + IFNULL(UnitsOnOrder, 0))
                 FROM Products
                 LIMIT 20;
                 ''')

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT ProductName, UnitPrice * (UnitsInStock + COALESCE(UnitsOnOrder, 0))
                 FROM Products
                 LIMIT 20;
                 ''')

# SQL Stored Procedure
- SQLite에서는 Stored Procedure를 지원하지 않음, SQLite는 어플리케이션이 직접 SQL문을 바로 실행시킬수 있기 때문
    - 다른 데이터베이스의 경우 어플리케이션과 DBMS가 분리되어 있는 경우가 있고, 여러 어플리케이션이 하나의 데이터베이스와 연길되는 경우가 많기 때문에 데이터베이스 단에서 자주 사용되는 쿼리들을 저장해둘 필요가 있다
    - 하지만 SQLite는 이를 사용하는 어플리케이션과 통합되기 때문에 굳이 Stored Procedure를 필요로 하지 않는다
        - SQLite와 통합된 어플리케이션은 여러 줄의 쿼리를 바로 보낼 수 있기 때문

출처: https://github.com/aergoio/sqlite-stored-procedures

In [ ]:
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers LIMIT 20;
                 ''')

In [ ]:
# stored procedure with one parameter
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers WHERE City=?;
                 ''',
                 'London',
                 )

In [ ]:
# stored porcedure with multiple parameters
execute_and_show(cur,
                 '''
                 SELECT * FROM Customers WHERE City=? AND PostalCode=?;
                 ''',
                 'London',
                 'WA1 1DP'
                 )